# Gap (prelaunch) 0.9 - July 2018
## NLP and CV Data Engineering Framework

<b>[Github] (https://github.com/andrewferlitsch/gap)</b>

# Automated PDF, Fax, Image Capture Text Extraction with Gap (Session 1)

Let's start with the basics. We will be using the <b style='color: saddlebrown'>SPLITTER</b> component in my Gap module.

Steps:
1. Import the <b style='color: saddlebrown'>Document</b> and <b style='color: saddlebrown'>Page</b> class from the <b style='color: saddlebrown'>splitter</b> module.
2. Create a Document object.
3. Pass a PDF (text or scanned), Facsimile (TIFF) or image captured document to the Document object.
4. Wait for the results :)

In [1]:
# let's go to the directory where Gap Framework is installed
import os
os.chdir("../")
!cd

C:\Users\'\Desktop\Gap-ml


In [2]:
# import Document and Page from the document module
from splitter import Document, Page

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\'\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\'\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## <span style='color: saddlebrown'>Document</span> Object

The initializer (constructor) takes the following arguments:<br/>

        document - path to the document
        dir      - directory where to store extracted pages and text
        ehandler - function to invoke when processing is completed in asynchronous mode
        config   - configuration settings for SYNTAX module
        
Let's start by preprocessing a 105 page PDF, which is a medical benefits plan. We should see:

- Split into individual PDF pages
- Text extracted from each page
- Individual page PDF and text stored in specified directory.


In [3]:
doc = Document("plan/nc.pdf", "plan/nc")

Ok, we are done! Let's look at a page, like page 105.

Wow, that's the foreign language translation page - see how it handles other (non-latin) character sets.

In [4]:
# Let's use the name property to see the name of the document
print( doc.name )

# Use the len() operator to find out how many pages are in the document
print( len(doc) )


nc
105


## <span style='color: saddlebrown'>Page</span> Object

Let's now dive deeper. When the document was processed, each page was put into a <b style='color: saddlebrown'>Page</b> object. Here are some things we can do:

1. Walk thru each page sequentially as an array index (list).<br/>
2. See the original text from the page.<br/>
3. See the "default" NLP preprocessing of the text on the page (which can be modified with config settings).<br/>


In [5]:
# Let's take a look at one of the pages
pages = doc.pages

# total number of pages
print(len(pages))

# Last page in the document
pages[104]

105


In [6]:
# Let's look at the text for that page (page 105)
page = pages[104]
page.text

'Legal Notices \n         Laotian    ໂປດຊາບ:  ຖ້າວ່າ  ທ່ານເວົ້າພາສາ  ລາວ,  ການບໍລິການຊ່ວຍ\n                    ເຫຼືອດ້ານພາສາ, ໂດຍບໍ່ເສັຽຄ່າ, ແມ່ນມີພ້ອມໃຫ້ທ່ານ. \n                    ໂທຣ 919-814-4400. \n         Japanese   注意事項：日本語を話される場合、無料の言語支援をご利用いただけます。919-814-\n                    4400. \n                           Notice of Grandfather Status \n     The State Health Plan believes the 70/30 Plan is a “grandfathered health plan” under the Patient Protection \n     and Affordable Care Act (the Affordable Care Act). As permitted by the Affordable Care Act, a grandfathered \n     health plan can preserve certain basic health coverage that was already in effect when that law was enacted. \n     Being a grandfathered health plan means that your plan may not include certain consumer protections of the \n     Affordable Care Act that apply to other plans, for example, the requirement for the provision of preventive \n     health services without any cost sharing. However, grandfathered hea

In [7]:
# Let's look at the default NLP preprocessing of the text (stemming, stopword removal, punct removal)
page.words


[{'tag': 0, 'word': 'legal'},
 {'tag': 0, 'word': 'ໂປດຊາບ'},
 {'tag': 0, 'word': 'ຖ'},
 {'tag': 0, 'word': 'າວ'},
 {'tag': 0, 'word': 'າ'},
 {'tag': 0, 'word': 'ທ'},
 {'tag': 0, 'word': 'ານເວ'},
 {'tag': 0, 'word': 'າພາສາ'},
 {'tag': 0, 'word': 'ລາວ'},
 {'tag': 0, 'word': 'ການບ'},
 {'tag': 0, 'word': 'ລ'},
 {'tag': 0, 'word': 'ການຊ'},
 {'tag': 0, 'word': 'ວຍ'},
 {'tag': 0, 'word': 'ເຫ'},
 {'tag': 0, 'word': 'ອດ'},
 {'tag': 0, 'word': 'ານພາສາ'},
 {'tag': 0, 'word': 'ໂດຍບ'},
 {'tag': 0, 'word': 'ເສ'},
 {'tag': 0, 'word': 'ຽຄ'},
 {'tag': 0, 'word': 'າ'},
 {'tag': 0, 'word': 'ແມ'},
 {'tag': 0, 'word': 'ນມ'},
 {'tag': 0, 'word': 'ພ'},
 {'tag': 0, 'word': 'ອມໃຫ'},
 {'tag': 0, 'word': 'ທ'},
 {'tag': 0, 'word': 'ານ'},
 {'tag': 0, 'word': 'ໂທຣ'},
 {'tag': 0, 'word': 'japanese'},
 {'tag': 0, 'word': '注意事項'},
 {'tag': 0, 'word': '日本語を話される場合'},
 {'tag': 0, 'word': '無料の言語支援をご利用いただけます'},
 {'tag': 0, 'word': 'notice'},
 {'tag': 0, 'word': 'believes'},
 {'tag': 0, 'word': 'grandfathered'},
 {'tag': 0,

We can see that some words appear a lot, like preventive, health and protection. Let's get information on the distribution of words in the page. There are two properties we can use for this purpose:

    freqDist - count of the number of occurrences of each word
    termFreq - percentage the word appears on the page (TF -> Term Frequency)

In [8]:
# Let's see the frequency distribution (word counts) for the page
page.freqDist

[('health', 8),
 ('plan', 8),
 ('grandfathered', 7),
 ('protections', 4),
 ('apply', 3),
 ('plans', 2),
 ('preventive', 2),
 ('າ', 2),
 ('share', 2),
 ('benefits', 2),
 ('cost', 2),
 ('ທ', 2),
 ('example', 2),
 ('consumer', 2),
 ('coverage', 1),
 ('healthcare', 1),
 ('contact', 1),
 ('າວ', 1),
 ('basic', 1),
 ('ແມ', 1),
 ('ໂປດຊາບ', 1),
 ('lifetime', 1),
 ('service', 1),
 ('日本語を話される場合', 1),
 ('directed', 1),
 ('currently', 1),
 ('ພ', 1),
 ('means', 1),
 ('ານພາສາ', 1),
 ('ານເວ', 1),
 ('cause', 1),
 ('preserve', 1),
 ('effect', 1),
 ('must', 1),
 ('legal', 1),
 ('無料の言語支援をご利用いただけます', 1),
 ('elimination', 1),
 ('change', 1),
 ('ຖ', 1),
 ('services', 1),
 ('ເຫ', 1),
 ('law', 1),
 ('ນມ', 1),
 ('ານ', 1),
 ('permit', 1),
 ('comply', 1),
 ('ວຍ', 1),
 ('questions', 1),
 ('ລາວ', 1),
 ('regard', 1),
 ('requirement', 1),
 ('provide', 1),
 ('ໂທຣ', 1),
 ('base', 1),
 ('ອດ', 1),
 ('enacted', 1),
 ('location', 1),
 ('ອມໃຫ', 1),
 ('s', 1),
 ('provision', 1),
 ('າພາສາ', 1),
 ('u', 1),
 ('japanese', 1),
 (

In [9]:
# Let's see the term frequency (TF)
page.termFreq

[('health', 0.07207207207207207),
 ('plan', 0.07207207207207207),
 ('grandfathered', 0.06306306306306306),
 ('protections', 0.036036036036036036),
 ('apply', 0.02702702702702703),
 ('plans', 0.018018018018018018),
 ('preventive', 0.018018018018018018),
 ('າ', 0.018018018018018018),
 ('share', 0.018018018018018018),
 ('benefits', 0.018018018018018018),
 ('cost', 0.018018018018018018),
 ('ທ', 0.018018018018018018),
 ('example', 0.018018018018018018),
 ('consumer', 0.018018018018018018),
 ('coverage', 0.009009009009009009),
 ('healthcare', 0.009009009009009009),
 ('contact', 0.009009009009009009),
 ('າວ', 0.009009009009009009),
 ('basic', 0.009009009009009009),
 ('ແມ', 0.009009009009009009),
 ('ໂປດຊາບ', 0.009009009009009009),
 ('lifetime', 0.009009009009009009),
 ('service', 0.009009009009009009),
 ('日本語を話される場合', 0.009009009009009009),
 ('directed', 0.009009009009009009),
 ('currently', 0.009009009009009009),
 ('ພ', 0.009009009009009009),
 ('means', 0.009009009009009009),
 ('ານພາສາ', 0.00

## <span style='color: saddlebrown'>Document</span> Object (Advanced)

Let's look at more advanced features of the Document object.

1. Word Count and Term Frequency
2. Save and Restore
3. Asychronous Processing of Documents
4. Scanned PDF / OCR

### Frequency Distribution

Let's look at a frequency distribution (word count) for the whole document. Note that if we look at just the top 10 word counts (after removing stopwords), it is very clear what the document is about: service, benefit, cover, health, medical, care, coverage, ...

If we look at the top 25 word counts, we can see secondary classification indicators, like: plan, medication, treatment, deductible, eligible, dependent, hospital, claim, authorization, prescription and limit.

HINT: It's a Healthcare Benefit Plan.

In [10]:
doc.freqDist

[('services', 529),
 ('provide', 426),
 ('covered', 332),
 ('health', 321),
 ('care', 302),
 ('benefit', 239),
 ('benefits', 221),
 ('network', 210),
 ('coverage', 207),
 ('information', 192),
 ('medical', 182),
 ('s', 158),
 ('plan', 146),
 ('treat', 142),
 ('certification', 129),
 ('deductible', 121),
 ('request', 112),
 ('medications', 111),
 ('review', 111),
 ('must', 110),
 ('eligible', 108),
 ('providers', 106),
 ('prior', 106),
 ('representative', 106),
 ('service', 101),
 ('members', 95),
 ('time', 93),
 ('hospital', 92),
 ('following', 92),
 ('authorization', 92),
 ('prescription', 88),
 ('available', 87),
 ('amount', 87),
 ('including', 85),
 ('days', 81),
 ('receive', 77),
 ('dependent', 74),
 ('bcbsnc', 72),
 ('period', 71),
 ('received', 70),
 ('facility', 69),
 ('emergency', 68),
 ('medication', 67),
 ('therapy', 67),
 ('license', 67),
 ('decision', 66),
 ('level', 66),
 ('www', 62),
 ('required', 62),
 ('responsible', 62),
 ('supplies', 62),
 ('date', 60),
 ('hours', 59)

### (Re) Load

When a Document object is created, the individual PDF pages, text extraction and NLP analysis are stored. 

The document can then be subsequently reloaded from storage without reprocessing.

In [11]:
# Let's first delete the Document object from memory
doc = None

In [12]:
# Let's reload the document from storage.
doc = Document()
doc.load("plan/nc.pdf", "plan/nc")

Let's show some examples of how the document was reconstructed from memory.

In [13]:
# Document Name, Number of Pages
print(doc.document)
print(len(doc))

plan/nc.pdf
105


In [14]:
# Let's print text from the last page
page = doc[104]
page.text

'                              Legal Notices \n         Laotian    ໂປດຊາບ:  ຖ້າວ່າ  ທ່ານເວົ້າພາສາ  ລາວ,  ການບໍລິການຊ່ວຍ\n                    ເຫຼືອດ້ານພາສາ, ໂດຍບໍ່ເສັຽຄ່າ, ແມ່ນມີພ້ອມໃຫ້ທ່ານ. \n                    ໂທຣ 919-814-4400. \n         Japanese   注意事項：日本語を話される場合、無料の言語支援をご利用いただけます。919-814-\n                    4400. \n                           Notice of Grandfather Status \n     The State Health Plan believes the 70/30 Plan is a “grandfathered health plan” under the Patient Protection \n     and Affordable Care Act (the Affordable Care Act). As permitted by the Affordable Care Act, a grandfathered \n     health plan can preserve certain basic health coverage that was already in effect when that law was enacted. \n     Being a grandfathered health plan means that your plan may not include certain consumer protections of the \n     Affordable Care Act that apply to other plans, for example, the requirement for the provision of preventive \n     health services without any cost shari

In [15]:
# Let's print the word (count) frequency distribution
doc.freqDist

[('services', 529),
 ('provide', 426),
 ('covered', 332),
 ('health', 321),
 ('care', 302),
 ('benefit', 239),
 ('benefits', 221),
 ('network', 210),
 ('coverage', 207),
 ('information', 192),
 ('medical', 182),
 ('s', 158),
 ('plan', 146),
 ('treat', 142),
 ('certification', 129),
 ('deductible', 121),
 ('request', 112),
 ('medications', 111),
 ('review', 111),
 ('must', 110),
 ('eligible', 108),
 ('providers', 106),
 ('prior', 106),
 ('representative', 106),
 ('service', 101),
 ('members', 95),
 ('time', 93),
 ('hospital', 92),
 ('following', 92),
 ('authorization', 92),
 ('prescription', 88),
 ('available', 87),
 ('amount', 87),
 ('including', 85),
 ('days', 81),
 ('receive', 77),
 ('dependent', 74),
 ('bcbsnc', 72),
 ('period', 71),
 ('received', 70),
 ('facility', 69),
 ('emergency', 68),
 ('medication', 67),
 ('therapy', 67),
 ('license', 67),
 ('decision', 66),
 ('level', 66),
 ('www', 62),
 ('required', 62),
 ('responsible', 62),
 ('supplies', 62),
 ('date', 60),
 ('hours', 59)

### Async Execution

Let's say you have PDF files arriving for processing in real-time from various sources. The ehandler option provides asynchronous processing of documents. When this option is specified, the document is processed on an independent process thread, and when complete the specified event handler is called.

In [16]:
def done(document):
    print("EVENT HANDLER: done")
    
doc = Document("plan/crash_2015.pdf", "plan/crash", ehandler=done)

EVENT HANDLER: done


Let's get a frequency distribution for this document. BTW, it is a 2015 State of Oregon table of crash statistics (single page) from a multi-page report. Note how the top ten words (after stopword removal) indicate what the document is about: serious, injury, fatal, crash, highway, death.

In [17]:
doc.freqDist

[('combine', 7),
 ('serious', 5),
 ('fatal', 4),
 ('inj', 4),
 ('injury', 4),
 ('system', 3),
 ('crash', 3),
 ('rate', 2),
 ('highway', 2),
 ('vmt', 2),
 ('miles', 2),
 ('vehicle', 1),
 ('classification', 1),
 ('injuries', 1),
 ('deate', 1),
 ('person', 1),
 ('v', 1),
 ('jurisdiction', 1),
 ('resulted', 1),
 ('ys', 1),
 ('table', 1),
 ('nog', 1),
 ('sustaining', 1),
 ('casualty', 1),
 ('lists', 1),
 ('total', 1),
 ('capable', 1),
 ('inon', 1),
 ('and', 1),
 ('drivinevents', 1),
 ('data', 1),
 ('prior', 1),
 ('wjury', 1),
 ('rmal', 1),
 ('injuriula', 1),
 ('activities', 1),
 ('crashes', 1),
 ('s', 1),
 ('es', 1),
 ('pralking', 1),
 ('functional', 1),
 ('death', 1),
 ('deaths', 1),
 ('annual', 1),
 ('continue', 1)]

### Scanned PDF / OCR

Let's now process a scanned PDF. That's a PDF which is effective a scanned image of a text document, which is then wrapped inside a PDF.

- Split into pages
- Extract page image
- OCR the image image into text
- Extract the text

In [19]:
Document.SCANCHECK = 0

# OCR the scanned PDF and extract text
doc = Document("plan/4scan.pdf", "plan/4scan")

Let's now look at a few properties of the preprocessed document

In [ ]:
# The scanned property indicates the document was a scanned PDF (true)
print( doc.scanned )

# Let's print the number of pages
print( len(doc) )

Let's now look at a page.

In [ ]:
# Get the first page
page = doc[0]

page.text

## END OF SESSION 1